# Weaviate Hybrid Search

This notebook shows how to use [Weaviate hybrid search](https://weaviate.io/blog/hybrid-search-explained) as a LangChain retriever.

In [1]:
import weaviate
import os

WEAVIATE_URL = "..."
client = weaviate.Client(
    url=WEAVIATE_URL,
)

In [2]:
from langchain.retrievers.weaviate_hybrid_search import WeaviateHybridSearchRetriever
from langchain.schema import Document

In [3]:
retriever = WeaviateHybridSearchRetriever(client, index_name="LangChain", text_key="text")

In [4]:
docs = [Document(page_content="foo")]

In [5]:
retriever.add_documents(docs)

['3f79d151-fb84-44cf-85e0-8682bfe145e0']

In [6]:
retriever.get_relevant_documents("foo")

[Document(page_content='foo', metadata={})]